In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import sys
import os

from metsim.methods import mtclim
from metsim import disaggregate, metsim, physics

In [ ]:
# Load custom functions
sys.path.append("./python_functions/")
from custom_data_process import convert_temp

In [ ]:
###############################################################################
#   Read data
###############################################################################
path_main = "./inputs/"
dir_input = os.path.join(path_main)

fname_temp_prec = "Delta1915_2015_4columns.txt"
fname_wind = "WP_Daily_1915_2015.txt"

dir_output = os.path.join("processing")
if not os.path.exists(dir_output):
  os.makedirs(dir_output)

data_temp_prec = np.loadtxt(os.path.join(dir_input, fname_temp_prec))
data_wind = np.loadtxt(os.path.join(dir_input, fname_wind))

In [ ]:
###############################################################################
#   Specify environment and parameters
###############################################################################
date_bgn = datetime(1915, 1, 1)
date_end = datetime(2015, 12, 31)

lat = 38
long = -121.5

# time step (in minutes)
time_step = 60

# meters
elev = 0

# daily precipitation (mm)
arr_prec_raw = data_temp_prec[:,0]

# max temperature (C)
arr_tmax_raw = data_temp_prec[:,1]

# min temperature (C)
arr_tmin_raw = data_temp_prec[:,2]

# wind speed (m/sec)
arr_wind_raw = data_wind

# Daily date_range
daterange_daily = pd.date_range(date_bgn, date_end)

# Account for actual date range
arr_prec_raw = arr_prec_raw[:len(daterange_daily)]
arr_tmax_raw = arr_tmax_raw[:len(daterange_daily)]
arr_tmin_raw = arr_tmin_raw[:len(daterange_daily)]
arr_wind_raw = arr_wind_raw[:len(daterange_daily)]


#
# For cloud cover fraction
#
#   smoothed daily temperature range using 30-day moving window
davg = 30

# Seasonal_precipitation requires past 90 days of data
dskip = 90

# Adjust date_bgn and other arrays to account for averaging
date_bgn_adj = date_bgn + timedelta(days = dskip)
arr_prec_adj = arr_prec_raw[dskip:]
arr_tmax_adj = arr_tmax_raw[dskip:]
arr_tmin_adj = arr_tmin_raw[dskip:]
arr_wind_adj = arr_wind_raw[dskip:]

# Update daily date_range
daterange_daily = pd.date_range(date_bgn_adj, date_end)

In [ ]:
# Model parameters. Default values are used following:
#   https://metsim.readthedocs.io/en/develop/configuration.html
#   https://metsim.readthedocs.io/en/1.1.0/configuration.html
params = {
    
    # The timestep to disaggregate in minutes. 
    # If given as 1440 (number of minutes in a day), 
    #   no disaggregation will occur. This value must divide 1440 evenly.
    "time_step": time_step,

    # The time to start simulation given in the format yyyy/mm/dd
    #   Hans: However, datetime seems to work as well.
    "start": date_bgn_adj,

    # The time to end simulation given in the format yyyy/mm/dd
    #   Hans: However, datetime seems to work as well.    
    "stop": date_end,
    
    # Scale factor for calculation of cloudy sky transmittance.
    # Defaults to 0.75, range should be between 0 and 1.
    "rain_scalar" : 0.75,

    #  Minimum precipitation threshold to take into account when
    #   simulating incoming shortwave radiation. Defaults to 0
    "sw_prec_thresh" : 0.0,
    
    # Type of cloud correction to longwave radiation to apply.
    # Can be either DEFAULT or CLOUD_DEARDORFF. Defaults to CLOUD_DEARDORFF.
    # Capitalization does not matter.
    'lw_cloud'    : "CLOUD_DEARDORFF",

    # Type of longwave radiation parameterization to apply.
    # Can be one of the following: DEFAULT, TVA, ANDERSON, BRUTSAERT, 
    #   SATTERLUND, IDSO, or PRATA. Defaults to PRATA.
    # Capitalization does not matter.
    "lw_type": "Default",
    
    # Type of precipitation disaggregation method to use.
    # Can be one of the following: uniform, triangle, or mix.
    # Defaults to uniform.
    "prec_type": "uniform",

    # Scale factor for calculation of daily mean temperature.
    # Defaults to 0.45, range should be between 0 and 1
    'tday_coef' : 0.45,
    
    # Used to calculate atmospheric pressure. Defaults to 0.0065 K/m.
    "lapse_rate" : 0.0065,
    
    # Convergence criteria for the iterative calculation of dewpoint temperature
    #   in MtClim. Defaults to 1e-6
    "tdew_tol" : 1e-6,
    
    # Whether to use UTC timecode offsets for shifting timeseries. 
    # Without this option all times should be considered local to the gridcell
    #   being processed. 
    # Large domain runs probably want to set this option to True
    "utc_offset": False,

    # Weight for calculation of time of maximum daily temperature. 
    # Must be between 0 and 1. Defaults to 0.67
    "tmax_daylength_fraction": 0.67, 

    # Flag to specify if output timesteps should be period-ending.
    # Default is period-beginning
    # https://metsim.readthedocs.io/en/develop/_modules/metsim/metsim.html
    "period_ending": False,

    # Not well documented, but needed for disaggregate
    # https://github.com/UW-Hydro/MetSim/blob/3550648f2f7452e69afc18a834cbb499846dc06b/metsim/datetime.py
    "calendar": "standard",

    # A string representing the simulation methods to use.
    # The current implementation only supports mtclim
    "method": "metclim",

    # Domain
    'elev': elev,
    'lat': lat,
    'lon': long
    }

In [ ]:
################################################################################
#   Compute cloud cover fraction
###############################################################################
#   daily temperature range
dtr0_raw = arr_tmax_raw - arr_tmin_raw
dtr0_adj = dtr0_raw[dskip:]

sm_dtr0 = np.zeros(len(dtr0_adj))
seasonal_prec0 = np.zeros(len(dtr0_adj))
ind = 0
for i in range(dskip, len(dtr0_raw)):
    
    # 30-day moving average
    sm_dtr0[ind] = np.average(dtr0_raw[i-davg:i])
    
    # precipitation over 90 days
    seasonal_prec0[ind] = np.sum(arr_prec_raw[i-dskip:i])
    ind = ind + 1

# Maximum daily transmittance of the atmosphere under cloudy conditions
tfmax0 = mtclim.tfmax(dtr0_adj, sm_dtr0, arr_prec_adj, params)

# Cloud cover fraction
tskc0 = mtclim.tskc(tfmax0, params)

In [ ]:
###############################################################################
#   Compute daily values in preparation for relative humidity
###############################################################################

# Obtain daylength, tt_max, and potrad for the study period.
# tiny_rad_fract_raw, daylength_raw, potrad_raw, tt_max_raw have 366  elements.
# Use a loop to assign a value to each day within the study period.
[tiny_rad_fract_raw, daylength_raw, potrad_raw, tt_max_raw] \
                                          = physics.solar_geom(elev, lat, long)

num_days = (date_end - date_bgn_adj).days + 1

assert (num_days == len(daterange_daily))
tiny_rad_fract0 = np.zeros([num_days, np.shape(tiny_rad_fract_raw)[1]])
daylength0 = np.zeros(num_days)
tt_max0 = np.zeros(num_days)
potrad0 = np.zeros(num_days)

for i in range(0, num_days):

    dstr0 = daterange_daily[i].strftime("%Y-%m-%d")
    iday = pd.Period(dstr0).dayofyear - 1

    tiny_rad_fract0[i, :] = tiny_rad_fract_raw[iday, :]
    daylength0[i] = daylength_raw[iday]
    tt_max0[i] = tt_max_raw[iday]
    potrad0[i] = potrad_raw[iday]

In [ ]:
###############################################################################
# Use the main function to obtain variables iteratively.
###############################################################################
df_daily = pd.DataFrame({"t_min": arr_tmin_adj,
                    "t_max": arr_tmax_adj,
                    "prec": arr_prec_adj,
                    "dtr": dtr0_adj,
                    "smoothed_dtr": sm_dtr0,
                    "tt_max": tt_max0,
                    "potrad": potrad0,
                    "daylength": daylength0,
                    "seasonal_prec": seasonal_prec0,
                    "wind": arr_wind_adj
                    })

df_daily = mtclim.run(df_daily, params)

In [ ]:
# Use date_range for index, as opposed to a numeric one.
df_daily = df_daily.set_index(daterange_daily)

In [ ]:
###############################################################################
# Disaggregate into sub-daily data
###############################################################################

# create dictionary version of solar_geom
solar_geom_dict = {"tiny_rad_fract": tiny_rad_fract0,
                   "daylength": daylength0,
                   "potrad": potrad0,
                   "tt_max": tt_max0}

df_disagg = disaggregate.disaggregate(df_daily, params, solar_geom_dict)

In [ ]:
df_disagg.head()

In [ ]:
###############################################################################
# Output result - Sub-daily data
###############################################################################
fname_out = os.path.join(dir_output, "Timeseries_step1_drybulb,raw_cloud,raw_relhum.csv")

fout = open(fname_out, "w")
fout.write("Date,Dry-bulb Temperature [F],Cloud_Cover [fraction],Relative Humidity [%]")
fout.write("\n")

daterange_sub = pd.date_range(date_bgn_adj
            , date_end + timedelta(days = 1) - timedelta(minutes = time_step),
            freq = timedelta(minutes = time_step))

npt = df_disagg.shape[0]
assert(len(daterange_sub) == npt)

temp_sub = df_disagg["temp"]
cloud_cover_sub = df_disagg["tskc"]
rel_humid_sub = df_disagg["rel_humid"]

for i in range(0, npt):
  dstr = daterange_sub[i].strftime("%Y-%m-%d %H:%M")
  fout.write(dstr + ","
            + "%.2f,"%(convert_temp(temp_sub[i], "c", "f"))
            + "%.2f,"%cloud_cover_sub[i]
            + "%.2f"%rel_humid_sub[i]            
            )
  fout.write("\n")

fout.close()